In [41]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
from keras.utils import to_categorical
import numpy as np
import os
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [85]:
mfcc_directory = 'data/mfcc_50/'
picklefiles = os.listdir(mfcc_directory)[:5]

ohe = OneHotEncoder(sparse=False)
le = LabelEncoder()

features = np.array([pickle.split(".")[0] for pickle in picklefiles])
fint = le.fit_transform(features).reshape(len(features),1)

ohe.fit(fint)

dataMap = {}

for picklefile in picklefiles:
    birdName = picklefile.split(".")[0]
    fullPath = os.path.join(mfcc_directory,picklefile)
    dataMap[birdName] = pickle.load(open(fullPath,'rb')).T

In [89]:
model = Sequential()

model.add(Bidirectional(LSTM(10, return_sequences=True), input_shape=(None, 50)))
model.add(Bidirectional(LSTM(10, return_sequences=True)))
model.add(Dense(5, activation = 'sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam')

In [98]:
def train_generator():
    while True:
        sequence_length = 115
        
        xarr = []
        yarr = []
        for k,v in dataMap.items():
            nsamples = v.shape[0]
            samples = np.random.randint(0,nsamples-sequence_length+1,size=100)
            birdohe = ohe.transform(le.transform([k]).reshape(1,1))
            for sample in samples:
                xarr.append(v[sample:sample+sequence_length,:].reshape(1,sequence_length,50))
                yarr.append(np.tile(birdohe,(sequence_length,1)).reshape(1,sequence_length,5))
        x_train = np.concatenate(xarr)
        y_train = np.concatenate(yarr)
        
#         x_train = np.random.random((1000, sequence_length, 5))
#         # y_train will depend on past 5 timesteps of x
#         y_train = x_train[:, :, 0]
#         for i in range(1, 5):
#             y_train[:, i:] += x_train[:, :-i, i]
#         y_train = to_categorical(y_train > 2.5)
        yield x_train, y_train

# x,y = train_generator()
history = model.fit_generator(train_generator(), steps_per_epoch=30, epochs=50, verbose=1)

plt.plot(history.history["loss"])
plt.show()

Epoch 1/50
30/30 [==============================] - 152s 5s/step - loss: 0.0355
Epoch 2/50
30/30 [==============================] - 144s 5s/step - loss: 0.0331
Epoch 3/50
30/30 [==============================] - 161s 5s/step - loss: 0.0369
Epoch 4/50
30/30 [==============================] - 158s 5s/step - loss: 0.0379
Epoch 5/50
30/30 [==============================] - 171s 6s/step - loss: 0.0360
Epoch 6/50
30/30 [==============================] - 162s 5s/step - loss: 0.0370
Epoch 7/50
30/30 [==============================] - 155s 5s/step - loss: 0.0356
Epoch 8/50
30/30 [==============================] - 159s 5s/step - loss: 0.0351
Epoch 9/50
30/30 [==============================] - 159s 5s/step - loss: 0.0333
Epoch 10/50
30/30 [==============================] - 164s 5s/step - loss: 0.0339
Epoch 11/50
30/30 [==============================] - 161s 5s/step - loss: 0.0337
Epoch 12/50
30/30 [==============================] - 156s 5s/step - loss: 0.0331
Epoch 13/50
30/30 [==================

In [95]:
plt.plot(history.history["loss"])
plt.show()

In [111]:
sequence_length = 50

xarr = []
yarr = []
for k,v in dataMap.items():
    nsamples = v.shape[0]
    samples = np.random.randint(0,nsamples-sequence_length+1,size=1)
    birdohe = np.array([0,0,0,0,0])
    for sample in samples:
        xarr.append(v[sample:sample+sequence_length,:].reshape(1,sequence_length,50))
        yarr.append(np.tile(birdohe,(sequence_length,1)).reshape(1,sequence_length,5))

x_train = np.concatenate(xarr)
y_train = np.concatenate(yarr)

model.predict(x_train[:2,:,:])[:,:50,:] 

ValueError: Error when checking input: expected bidirectional_31_input to have shape (115, 50) but got array with shape (50, 50)

In [99]:
mfcc_directory = 'data/mfcc_50/'
picklefiles = os.listdir(mfcc_directory)[-5:]

dataMap2 = {}

for picklefile in picklefiles:
    birdName = picklefile.split(".")[0]
    fullPath = os.path.join(mfcc_directory,picklefile)
    dataMap2[birdName] = pickle.load(open(fullPath,'rb')).T